# Data Extraction

It includes the following tasks

*   Reading the data from website in csv / json .
*   extracting useful information using exploratory analysis and then cleaning the data.



In [ ]:
# # Questions Written
# !wget http://data.assemblee-nationale.fr/static/openData/repository/15/questions/questions_ecrites/Questions_ecrites_XV.json.zip
# # Questions Asked to Governement 
# !wget http://data.assemblee-nationale.fr/static/openData/repository/15/questions/questions_gouvernement/Questions_gouvernement_XV.json.zip
# # Questions Orales
# !wget http://data.assemblee-nationale.fr/static/openData/repository/15/questions/questions_orales_sans_debat/Questions_orales_sans_debat_XV.json.zip
# # Unzipping All the Files
# !unzip Questions_ecrites_XV.json.zip
# !unzip Questions_gouvernement_XV.json.zip
# !unzip Questions_orales_sans_debat_XV.json.zip

In [ ]:
import os
import numpy as np
import pandas as pd
import json
import re
from bs4 import BeautifulSoup

Fetching the Json Files


In [ ]:
from os import walk 
files = [] 
foldersToCheck = ['json/']
while (len(foldersToCheck) > 0): 
  for (dirpath, dirnames, filenames) in walk(foldersToCheck[0]): 
    while(len(dirnames) > 0): 
        foldersToCheck.append(foldersToCheck[0] + dirnames[0] + "/") 
        del dirnames[0] 
    while(len(filenames) > 0): 
        files.append(foldersToCheck[0] + filenames[0]) 
        del filenames[0] 
    del foldersToCheck[0] 
# print(len(files)) 

In [ ]:


path_to_json = 'drive/My Drive/json_normal/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]


80


Extracting the ID to match

In [ ]:
questions_df = pd.DataFrame()
extracted_data = []
for index, js in enumerate(json_files):
  with open(os.path.join(path_to_json, js)) as json_file:
      json_text = json.load(json_file)
      extracted_data.append(json_text)
      questions_df = questions_df.append(json_text.get('question'),ignore_index=True)


Index(['@xmlns', '@xmlns:xsi', '@xsi:type', 'auteur', 'cloture', 'identifiant',
       'indexationAN', 'minAttribs', 'minInt', 'textesReponse', 'type', 'uid'],
      dtype='object')
{'question': {'@xmlns': 'http://schemas.assemblee-nationale.fr/referentiel', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xsi:type': 'QuestionGouvernement_Type', 'uid': 'QANR5L15QG1050', 'identifiant': {'numero': '1050', 'regime': '5eme R�publique', 'legislature': '15'}, 'type': 'QG', 'indexationAN': {'rubrique': 'environnement', 'teteAnalyse': None, 'ANALYSE': {'ANA': "inscription de la protection de l'environnement à l'article 1er de la Constitution"}}, 'auteur': {'identite': {'acteurRef': 'PA608695', 'mandatRef': 'PM723564'}, 'groupe': {'organeRef': 'PO730946', 'abrege': 'NG', 'developpe': 'Nouvelle Gauche'}}, 'minInt': {'abrege': 'Transition écologique et solidaire', 'developpe': 'Ministère de la transition écologique et solidaire'}, 'minAttribs': {'minAttrib': {'denomination': {'abrege'

Manipluating the value for matching and chnaging the data types

In [ ]:
# def get_ref_values():
sv = []
new_dict = {}
for i in range(len(questions_df)):
  u = ''.join([x for x in list(questions_df['auteur'].loc[i].get('identite').get('acteurRef')) if not x.isalpha()])
  sv.append(u)
questions_df['auteur'] = sv
questions_df['auteur'] = questions_df['auteur'].astype('int64')


In [ ]:
for i in range(len(questions_df)):
  questions_df['DateCloture'] = questions_df['cloture'].loc[i]['dateCloture']
  questions_df['question_motive'] = questions_df['indexationAN'].loc[i]['ANALYSE']['ANA']
  questions_df['question_response'] = questions_df['textesReponse'].loc[i]['texteReponse']['texte']
  if questions_df.get('textesQuestion',0):
    questions_df['question_asked'] : questions_df['textesQuestion'].loc[i]['texteQuestion']['texte']
  else:
    questions_df['question_asked'] = ''

Extracting the data from the Nodeputes file

In [ ]:
nodeputes_df = pd.read_csv('/content/drive/My Drive/nosdeputes.fr_synthese_2020-10-02.csv',encoding='utf-8',sep=';',error_bad_lines=False)
nodeputes_df.head()


,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,id_an,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
0,32,Damien Abad,Abad,Damien,H,1980-04-05,Nîmes (Gard),01,Ain,5,2017-06-21,NaN,0,LR,Les Républicains,http://www.damienabad.fr|https://twitter.com/d...,NaN,NaN,0,85,http://www2.assemblee-nationale.fr/deputes/fic...,605036,damien-abad,https://www.nosdeputes.fr/damien-abad,https://www.nosdeputes.fr/damien-abad/csv,1,damienabad,40,49,22,306,349,134,3012,105,0,0,38,37,8
1,43,Caroline Abadie,Abadie,Caroline,F,1976-09-07,Saint-Martin-d'Hères (Isère),38,Isère,8,2017-06-21,NaN,0,LREM,La République en Marche,https://twitter.com/AbadieCaroline|https://fr-...,NaN,NaN,Industriel-Chef entreprise,200,http://www2.assemblee-nationale.fr/deputes/fic...,719866,caroline-abadie,https://www.nosdeputes.fr/caroline-abadie,https://www.nosdeputes.fr/caroline-abadie/csv,1,AbadieCaroline,33,66,18,21,8,14,335,189,0,0,4,3,0
2,493,Jean-Félix Acquaviva,Acquaviva,Jean-Félix,H,1973-03-19,Bastia (Haute-Corse),2B,Haute-Corse,2,2017-06-21,NaN,0,LT,Régions et peuples solidaires,https://fr-fr.facebook.com/jeanfelixacquaviva/...,NaN,NaN,Cadre,650,http://www2.assemblee-nationale.fr/deputes/fic...,719146,jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva...,1,JF_Acquaviva,25,33,48,121,11,188,2444,91,0,0,7,22,2
3,152,Lénaïck Adam,Adam,Lénaïck,H,1992-02-19,Saint Laurent du Maroni (Guyane),973,Guyane,2,2018-03-12,NaN,0,LREM,La République en Marche,https://fr-fr.facebook.com/lenaick.adam/|https...,NaN,NaN,Cadre supérieur (entreprises privée),357,http://www2.assemblee-nationale.fr/deputes/fic...,721036,lenaick-adam,https://www.nosdeputes.fr/lenaick-adam,https://www.nosdeputes.fr/lenaick-adam/csv,1,LenaickADAM,10,4,5,3,0,9,516,171,1,0,5,1,0
4,234,Damien Adam,Adam,Damien,H,1989-06-28,Orléans (Loiret),76,Seine-Maritime,1,2017-06-21,NaN,0,LREM,La République en Marche,https://twitter.com/damienadam76|http://damien...,NaN,NaN,Cadre supérieur (secteur privé),319,http://www2.assemblee-nationale.fr/deputes/fic...,722038,damien-adam,https://www.nosdeputes.fr/damien-adam,https://www.nosdeputes.fr/damien-adam/csv,1,damienadam76,29,93,167,15,4,69,663,222,0,0,5,8,2


Replacing the name to merge both files

In [ ]:
# Rename the column values
nodeputes_df.rename(columns={'id_an':'auteur'},inplace=True)
nodeputes_df.head()

,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,auteur,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
0,32,Damien Abad,Abad,Damien,H,1980-04-05,Nîmes (Gard),01,Ain,5,2017-06-21,NaN,0,LR,Les Républicains,http://www.damienabad.fr|https://twitter.com/d...,NaN,NaN,0,85,http://www2.assemblee-nationale.fr/deputes/fic...,605036,damien-abad,https://www.nosdeputes.fr/damien-abad,https://www.nosdeputes.fr/damien-abad/csv,1,damienabad,40,49,22,306,349,134,3012,105,0,0,38,37,8
1,43,Caroline Abadie,Abadie,Caroline,F,1976-09-07,Saint-Martin-d'Hères (Isère),38,Isère,8,2017-06-21,NaN,0,LREM,La République en Marche,https://twitter.com/AbadieCaroline|https://fr-...,NaN,NaN,Industriel-Chef entreprise,200,http://www2.assemblee-nationale.fr/deputes/fic...,719866,caroline-abadie,https://www.nosdeputes.fr/caroline-abadie,https://www.nosdeputes.fr/caroline-abadie/csv,1,AbadieCaroline,33,66,18,21,8,14,335,189,0,0,4,3,0
2,493,Jean-Félix Acquaviva,Acquaviva,Jean-Félix,H,1973-03-19,Bastia (Haute-Corse),2B,Haute-Corse,2,2017-06-21,NaN,0,LT,Régions et peuples solidaires,https://fr-fr.facebook.com/jeanfelixacquaviva/...,NaN,NaN,Cadre,650,http://www2.assemblee-nationale.fr/deputes/fic...,719146,jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva...,1,JF_Acquaviva,25,33,48,121,11,188,2444,91,0,0,7,22,2
3,152,Lénaïck Adam,Adam,Lénaïck,H,1992-02-19,Saint Laurent du Maroni (Guyane),973,Guyane,2,2018-03-12,NaN,0,LREM,La République en Marche,https://fr-fr.facebook.com/lenaick.adam/|https...,NaN,NaN,Cadre supérieur (entreprises privée),357,http://www2.assemblee-nationale.fr/deputes/fic...,721036,lenaick-adam,https://www.nosdeputes.fr/lenaick-adam,https://www.nosdeputes.fr/lenaick-adam/csv,1,LenaickADAM,10,4,5,3,0,9,516,171,1,0,5,1,0
4,234,Damien Adam,Adam,Damien,H,1989-06-28,Orléans (Loiret),76,Seine-Maritime,1,2017-06-21,NaN,0,LREM,La République en Marche,https://twitter.com/damienadam76|http://damien...,NaN,NaN,Cadre supérieur (secteur privé),319,http://www2.assemblee-nationale.fr/deputes/fic...,722038,damien-adam,https://www.nosdeputes.fr/damien-adam,https://www.nosdeputes.fr/damien-adam/csv,1,damienadam76,29,93,167,15,4,69,663,222,0,0,5,8,2


Merging the files and converting it into CSV

In [ ]:
#  Concatenating two dataframes

data_df = pd.merge(questions_df, nodeputes_df, on="auteur")
data_df.to_csv('Questions.csv')

                                               @xmlns  ... questions_orales
0   http://schemas.assemblee-nationale.fr/referentiel  ...                9
1   http://schemas.assemblee-nationale.fr/referentiel  ...                0
2   http://schemas.assemblee-nationale.fr/referentiel  ...                1
3   http://schemas.assemblee-nationale.fr/referentiel  ...                1
4   http://schemas.assemblee-nationale.fr/referentiel  ...                0
..                                                ...  ...              ...
70  http://schemas.assemblee-nationale.fr/referentiel  ...                7
71  http://schemas.assemblee-nationale.fr/referentiel  ...                2
72  http://schemas.assemblee-nationale.fr/referentiel  ...                8
73  http://schemas.assemblee-nationale.fr/referentiel  ...                5
74  http://schemas.assemblee-nationale.fr/referentiel  ...                2

[75 rows x 55 columns]


In [ ]:
df_question = pd.read_csv('/content/Questions.csv')


In [ ]:
df_question.rename(columns={df_question.columns[0]: "Number" , df_question.columns[1]:"URL" , df_question.columns[2]:"URI","@xsi:type":"Type"},inplace=True)
df_question.set_index("Number")

,URL,URI,Type,auteur,cloture,identifiant,indexationAN,minAttribs,minInt,textesReponse,type,uid,DateCloture,question_motive,question_response,question_asked,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
Number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QuestionGouvernement_Type,608695,"{'codeCloture': 'REP_PUB', 'libelleCloture': '...","{'numero': '1050', 'regime': '5eme R�publique'...","{'rubrique': 'environnement', 'teteAnalyse': N...",{'minAttrib': {'denomination': {'abrege': 'Tra...,{'abrege': 'Transition écologique et solidaire...,{'texteReponse': {'infoJO': {'typeJO': 'JO_DEB...,QG,QANR5L15QG1050,21/06/2018,Union européenne,"</p><p align=""CENTER""> UNION EUROPÉENNE <a nam...",NaN,429,Cécile Untermaier,Untermaier,Cécile,F,1951-12-28,Belley (Ain),71,Saône-et-Loire,4,2017-06-21,NaN,0,SOC,Parti socialiste,https://twitter.com/ceciluntermaier|https://fr...,NaN,NaN,Fonctionnaire des grands corps de l'État,580,http://www2.assemblee-nationale.fr/deputes/fic...,cecile-untermaier,https://www.nosdeputes.fr/cecile-untermaier,https://www.nosdeputes.fr/cecile-untermaier/csv,1,ceciluntermaier,35,85,109,173,170,246,3642,180,0,0,11,40,9
1,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QuestionGouvernement_Type,719570,"{'codeCloture': 'REP_PUB', 'libelleCloture': '...","{'numero': '1014', 'regime': '5eme R�publique'...",{'rubrique': 'formation professionnelle et app...,{'minAttrib': {'denomination': {'abrege': 'Tra...,"{'abrege': 'Travail', 'developpe': 'Ministère ...",{'texteReponse': {'infoJO': {'typeJO': 'JO_DEB...,QG,QANR5L15QG1014,21/06/2018,Union européenne,"</p><p align=""CENTER""> UNION EUROPÉENNE <a nam...",NaN,365,Catherine Fabre,Fabre,Catherine,F,1978-09-19,Toulouse (Haute-Garonne),33,Gironde,2,2017-06-21,NaN,0,LREM,La République en Marche,http://catherinefabre.fr|https://twitter.com/c...,NaN,NaN,Maître de conférences des Universités,217,http://www2.assemblee-nationale.fr/deputes/fic...,catherine-fabre,https://www.nosdeputes.fr/catherine-fabre,https://www.nosdeputes.fr/catherine-fabre/csv,1,cfabreAN,29,60,33,103,14,45,428,207,0,0,5,0,0
2,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QuestionGouvernement_Type,719994,"{'codeCloture': 'REP_PUB', 'libelleCloture': '...","{'numero': '1021', 'regime': '5eme R�publique'...","{'rubrique': 'aménagement du territoire', 'tet...",{'minAttrib': {'denomination': {'abrege': 'Coh...,"{'abrege': 'Cohésion des territoires', 'develo...",{'texteReponse': {'infoJO': {'typeJO': 'JO_DEB...,QG,QANR5L15QG1021,21/06/2018,Union européenne,"</p><p align=""CENTER""> UNION EUROPÉENNE <a nam...",NaN,245,Catherine Kamowski,Kamowski,Catherine,F,1958-04-08,Valenciennes (Nord),38,Isère,5,2017-06-21,NaN,0,LREM,La République en Marche,http://catherinekamowski.fr|https://twitter.co...,NaN,NaN,Professeur du secondaire et technique,335,http://www2.assemblee-nationale.fr/deputes/fic...,catherine-kamowski,https://www.nosdeputes.fr/catherine-kamowski,https://www.nosdeputes.fr/catherine-kamowski/csv,2,CKamowski3805,26,48,17,5,5,11,491,227,0,0,7,1,1
3,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QuestionGouvernement_Type,721608,"{'codeCloture': 'REP_PUB', 'libelleCloture': '...","{'numero': '1058', 'regime': '5eme R�publique'...","{'rubrique': 'politique économique', 'teteAnal..

Removing Unnecessary Columns

In [ ]:
df_question.drop(columns=['indexationAN','textesReponse','auteur','cloture','minAttribs','minInt','identifiant','Type'],axis=1,inplace=True)

Cleaning the Data inside

In [ ]:
df_question['DateCloture'] = pd.to_datetime(df_question['DateCloture'])
df_question['date_naissance'] = pd.to_datetime(df_question['date_naissance'])
df_question['question_response'] = df_question['question_response'].str.replace(r'</p><p align="CENTER">', '')
df_question['question_response'] = df_question['question_response'].str.replace('<a name=PG1></a> </p><br>','')
df_question['question_response'] = df_question['question_response'].str.replace('<strong>','')
df_question['question_response'] = df_question['question_response'].str.replace('</strong>','')
df_question['question_response'] = df_question['question_response'].str.replace('<br><br>','')
df_question['question_response'] = df_question['question_response'].str.replace('<i>','')
df_question['question_response'] = df_question['question_response'].str.replace('</i>','')
df_question['question_response'] = df_question['question_response'].str.replace('</i><br><br>','')
df_question['question_response'] = df_question['question_response'].str.replace('<br> <p>','')


 UNION EUROPÉENNE M. le président. La parole est à M. Sylvain Waserman, pour le groupe du Mouvement démocrate et apparentés.M. Sylvain Waserman. Monsieur le secrétaire d'État auprès du ministre de l'Europe et des affaires étrangères, hier, le Président de la République et la Chancelière allemande se sont rencontrés pour un nouveau sommet franco-allemand à dix jours d'un Conseil européen déterminant pour l'avenir de l'Europe, déterminant parce que, en cette période troublée par des tensions multiples, elle doit faire la preuve qu'elle sait se réinventer et innover.Le couple franco-allemand peut et doit avoir un rôle d'impulsion déterminant dans deux domaines, le premier étant le défi majeur que représente la crise migratoire. Il n'existe pas un seul jour sans que l'actualité ne résonne comme un signal d'alarme face à ce défi auquel il nous faut trouver une réponse résolument européenne. En cette Journée mondiale des réfugiés, et alors que ce sujet fait l'actualité en Allemagne, des ulti

In [ ]:
df_question.to_csv('New_Questions.csv')

# Text Summarization

Here we got the extracted and cleaned data we required for text classification and sentiment -analysis.

Before approcahing towards sentiment analysis , we will summarize the data to find out the actual data to be used in sentiment analysis task.



In [ ]:
ques_df = pd.read_csv('New_Questions.csv')
ques_df.head()

,Unnamed: 0,Number,URL,URI,type,uid,DateCloture,question_motive,question_response,question_asked,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
0,0,0,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1050,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,429,Cécile Untermaier,Untermaier,Cécile,F,1951-12-28,Belley (Ain),71,Saône-et-Loire,4,2017-06-21,NaN,0,SOC,Parti socialiste,https://twitter.com/ceciluntermaier|https://fr...,NaN,NaN,Fonctionnaire des grands corps de l'État,580,http://www2.assemblee-nationale.fr/deputes/fic...,cecile-untermaier,https://www.nosdeputes.fr/cecile-untermaier,https://www.nosdeputes.fr/cecile-untermaier/csv,1,ceciluntermaier,35,85,109,173,170,246,3642,180,0,0,11,40,9
1,1,1,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1014,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,365,Catherine Fabre,Fabre,Catherine,F,1978-09-19,Toulouse (Haute-Garonne),33,Gironde,2,2017-06-21,NaN,0,LREM,La République en Marche,http://catherinefabre.fr|https://twitter.com/c...,NaN,NaN,Maître de conférences des Universités,217,http://www2.assemblee-nationale.fr/deputes/fic...,catherine-fabre,https://www.nosdeputes.fr/catherine-fabre,https://www.nosdeputes.fr/catherine-fabre/csv,1,cfabreAN,29,60,33,103,14,45,428,207,0,0,5,0,0
2,2,2,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1021,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,245,Catherine Kamowski,Kamowski,Catherine,F,1958-04-08,Valenciennes (Nord),38,Isère,5,2017-06-21,NaN,0,LREM,La République en Marche,http://catherinekamowski.fr|https://twitter.co...,NaN,NaN,Professeur du secondaire et technique,335,http://www2.assemblee-nationale.fr/deputes/fic...,catherine-kamowski,https://www.nosdeputes.fr/catherine-kamowski,https://www.nosdeputes.fr/catherine-kamowski/csv,2,CKamowski3805,26,48,17,5,5,11,491,227,0,0,7,1,1
3,3,3,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1058,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,171,Jean-Louis Bourlanges,Bourlanges,Jean-Louis,H,1946-07-13,Neuilly-Sur-Seine (Hauts-de-Seine),92,Hauts-de-Seine,12,2017-06-21,NaN,0,MODEM,Mouvement Démocrate,NaN,NaN,NaN,Retraité de la fonction publique,165,http://www2.assemblee-nationale.fr/deputes/fic...,jean-louis-bourlanges,https://www.nosdeputes.fr/jean-louis-bourlanges,https://www.nosdeputes.fr/jean-louis-bourlange...,1,NaN,33,65,84,14,17,14,719,156,0,0,2,0,1
4,4,4,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1051,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,104,Christine Hennion,Hennion,Christine,F,1955-08-23,Roubaix (Nord),92,Hauts-de-Seine,3,2017-06-21,NaN,0,LREM,La République en Marche,http://www.christine-hennion.fr|https://twitte...,NaN,NaN,Cadre supérieur (secteur privé),239,http://www2.assemblee-nationale.fr/deputes/fic...,christine-hennion,https://www.nosdeputes.fr/christine-hennion,https://www.nosdeputes.fr/christine-hennion/csv,1,Ch_Hennion,23,44,25,15,3,38,447,195,1,0,4,0,0


In [ ]:

!pip install -q transformers


     |████████████████████████████████| 1.8MB 7.8MB/s 
     |████████████████████████████████| 2.9MB 37.9MB/s 
     |████████████████████████████████| 890kB 57.2MB/s 


In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline




# Sentiment Analysis

With the help of transformers library of NLP , we manage to apply sentiment analysis and to get the polarity of each of the question asked in the statement.

In [ ]:
nlp_sentence_classif = pipeline('sentiment-analysis')


In [ ]:
ques_df['question_response']

0      UNION EUROPÉENNE M. le président. La parole e...
1      UNION EUROPÉENNE M. le président. La parole e...
2      UNION EUROPÉENNE M. le président. La parole e...
3      UNION EUROPÉENNE M. le président. La parole e...
4      UNION EUROPÉENNE M. le président. La parole e...
                            ...                        
70     UNION EUROPÉENNE M. le président. La parole e...
71     UNION EUROPÉENNE M. le président. La parole e...
72     UNION EUROPÉENNE M. le président. La parole e...
73     UNION EUROPÉENNE M. le président. La parole e...
74     UNION EUROPÉENNE M. le président. La parole e...
Name: question_response, Length: 75, dtype: object

In [ ]:
import pandas as pd

from collections import Counter
scores = []
nlp_sentence_classif = pipeline('sentiment-analysis')

for i in range(50):
  labs = nlp_sentence_classif(list(ques_df.iloc[i]['question_response']))
  scores.append([dict(Counter(["POS" if text.get('label') == "POSITIVE" else "NEG" for text in labs])), ques_df.iloc[i]['question_response']])



In [ ]:
def statement_polarity(pos , neg):
  if pos == neg:
    return "NEUTRAL"
  elif pos > neg:
    return "POSITIVE STATEMENT"
  else:
    return "NEGATIVE STATEMENT"


In [ ]:
ques_df['statement_type'] = ques_df.apply(lambda x: statement_polarity(x['POS'], x['NEG']), axis=1)

In [ ]:
ques_df[['POS','NEG']] = pd.DataFrame([sco[0].values() for sco in scores])
ques_df['POS'] = pd.to_numeric(ques_df['POS'], errors='coerce')
ques_df['NEG'] = pd.to_numeric(ques_df['NEG'], errors='coerce')
ques_df = ques_df.dropna(subset=['POS'])
ques_df = ques_df.dropna(subset=['NEG'])
ques_df['POS'] = ques_df['POS'].astype(int)
ques_df['NEG'] = ques_df['NEG'].astype(int)
ques_df.head()

,Unnamed: 0,Number,URL,URI,type,uid,DateCloture,question_motive,question_response,question_asked,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales,POS,NEG
0,0,0,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1050,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,429,Cécile Untermaier,Untermaier,Cécile,F,1951-12-28,Belley (Ain),71,Saône-et-Loire,4,2017-06-21,NaN,0,SOC,Parti socialiste,https://twitter.com/ceciluntermaier|https://fr...,NaN,NaN,Fonctionnaire des grands corps de l'État,580,http://www2.assemblee-nationale.fr/deputes/fic...,cecile-untermaier,https://www.nosdeputes.fr/cecile-untermaier,https://www.nosdeputes.fr/cecile-untermaier/csv,1,ceciluntermaier,35,85,109,173,170,246,3642,180,0,0,11,40,9,3818,398
1,1,1,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1014,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,365,Catherine Fabre,Fabre,Catherine,F,1978-09-19,Toulouse (Haute-Garonne),33,Gironde,2,2017-06-21,NaN,0,LREM,La République en Marche,http://catherinefabre.fr|https://twitter.com/c...,NaN,NaN,Maître de conférences des Universités,217,http://www2.assemblee-nationale.fr/deputes/fic...,catherine-fabre,https://www.nosdeputes.fr/catherine-fabre,https://www.nosdeputes.fr/catherine-fabre/csv,1,cfabreAN,29,60,33,103,14,45,428,207,0,0,5,0,0,3818,398
2,2,2,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1021,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,245,Catherine Kamowski,Kamowski,Catherine,F,1958-04-08,Valenciennes (Nord),38,Isère,5,2017-06-21,NaN,0,LREM,La République en Marche,http://catherinekamowski.fr|https://twitter.co...,NaN,NaN,Professeur du secondaire et technique,335,http://www2.assemblee-nationale.fr/deputes/fic...,catherine-kamowski,https://www.nosdeputes.fr/catherine-kamowski,https://www.nosdeputes.fr/catherine-kamowski/csv,2,CKamowski3805,26,48,17,5,5,11,491,227,0,0,7,1,1,3818,398
3,3,3,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1058,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,171,Jean-Louis Bourlanges,Bourlanges,Jean-Louis,H,1946-07-13,Neuilly-Sur-Seine (Hauts-de-Seine),92,Hauts-de-Seine,12,2017-06-21,NaN,0,MODEM,Mouvement Démocrate,NaN,NaN,NaN,Retraité de la fonction publique,165,http://www2.assemblee-nationale.fr/deputes/fic...,jean-louis-bourlanges,https://www.nosdeputes.fr/jean-louis-bourlanges,https://www.nosdeputes.fr/jean-louis-bourlange...,1,NaN,33,65,84,14,17,14,719,156,0,0,2,0,1,3818,398
4,4,4,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,QG,QANR5L15QG1051,2018-06-21,Union européenne,UNION EUROPÉENNE M. le président. La parole e...,NaN,104,Christine Hennion,Hennion,Christine,F,1955-08-23,Roubaix (Nord),92,Hauts-de-Seine,3,2017-06-21,NaN,0,LREM,La République en Marche,http://www.christine-hennion.fr|https://twitte...,NaN,NaN,Cadre supérieur (secteur privé),239,http://www2.assemblee-nationale.fr/deputes/fic...,christine-hennion,https://www.nosdeputes.fr/christine-hennion,https://www.nosdeputes.fr/christine-hennion/csv,1,Ch_Hennion,23,44,25,15,3,38,447,195,1,0,4,0,0,3818,398
